In [1]:
9import pandas as pd

In [12]:
original_data = pd.read_csv('ResourceEstimates.csv')
original_data.head()

,SiteCode,ShortTitle,ResourceCode,StartDate,EndDate,ReportingStandard,JORCCategory,JORCStatus,IN_TOTAL_FLAG,In Total Type,...,ContainedMetal,ContainedMetalUnit,CutOffGrade,CutOffGradeUnit,HeavyMineralPct,TargetCommodityGroup,Site_MineralizationStyle,Site_SecondaryMineralizationStyle,Resource_MineralizationStyle,Resource_SecondaryMineralizationStyle
0,S0000001,Pieces of Eight,1,13/07/2005,14/09/2006,Unknown,Resource,Inferred,No,Not in Total,...,903.50,kilogram,NaN,gram/tonne,NaN,GOLD,Vein and hydrothermal - undivided,Unknown/Unassigned,Unknown/Unassigned,Unknown/Unassigned
1,S0000002,Albury Heath,2,30/06/1999,01/10/2003,Unknown,Resource,Indicated,No,Not in Total,...,379.48,kilogram,NaN,gram/tonne,NaN,GOLD,Vein and hydrothermal - undivided,Au in shears or faults,Unknown/Unassigned,Au in shears or faults
2,S0000002,Albury Heath,3,04/06/2003,06/02/2017,Unknown,Resource,Measured,Yes,Resources,...,108.16,kilogram,NaN,gram/tonne,NaN,GOLD,Vein and hydrothermal - undivided,Au in shears or faults,Unknown/Unassigned,Unknown/Unassigned
3,S0000002,Albury Heath,4,04/06/2003,06/02/2017,Unknown,Resource,Indicated,Yes,Resources,...,361.68,kilogram,NaN,gram/tonne,NaN,GOLD,Vein and hydrothermal - undivided,Au in shears or faults,Unknown/Unassigned,Unknown/Unassigned
4,S0000002,Albury Heath,28452,07/02/2017,11/03/2019,"JORC (Australia, version unknown)",Resource,Indicated,Yes,Resources,...,675.00,kilogram,0.5,gram/tonne,NaN,GOLD,Vein and hydrothermal - undivided,Au in shears or faults,Unknown/Unassigned,Unknown/Unassigned


In [13]:
original_data.columns

Index(['SiteCode', 'ShortTitle', 'ResourceCode', 'StartDate', 'EndDate',
       'ReportingStandard', 'JORCCategory', 'JORCStatus', 'IN_TOTAL_FLAG',
       'In Total Type', 'Tonnage (Mt)', 'Grade', 'GradeUnit', 'CommodityName',
       'ContainedMetal', 'ContainedMetalUnit', 'CutOffGrade',
       'CutOffGradeUnit', 'HeavyMineralPct', 'TargetCommodityGroup',
       'Site_MineralizationStyle', 'Site_SecondaryMineralizationStyle',
       'Resource_MineralizationStyle',
       'Resource_SecondaryMineralizationStyle'],
      dtype='object')

In [14]:
df=original_data[['SiteCode','StartDate', 'EndDate','Tonnage (Mt)','Grade','GradeUnit','CommodityName','ContainedMetal','ContainedMetalUnit','CutOffGrade','TargetCommodityGroup']]

In [15]:
df['ContainedMetalUnit'].value_counts()

kilotonne         21614
kilogram          20076
million tonnes    12381
tonnes             3234
million carats      215
000 tonnes           13
Name: ContainedMetalUnit, dtype: int64

In [16]:
df['GradeUnit'].value_counts()

percent           36869
gram/tonne        20171
kilogram/tonne      278
carat/tonne         215
Name: GradeUnit, dtype: int64

## Convert unit measures to tonne unit

* 000 tonnes - long ton (Brit) a unit of weight equal to 2240 pounds or 1016.046909 kilograms = 1.016.
* million carats = 0.2 tonne
* million tonnes = 1 000 000 tonnes
* kilotonne = 1000  tonnes
* kilogram = 0.001 tonnes

In [17]:
quantity=[]
for i in range(len(df)):
    if df.iloc[i,8] == 'kilotonne':
        quantity.append(df.iloc[i,7]*1000)
    elif df.iloc[i,8] == 'kilogram':
        quantity.append(df.iloc[i,7]/1000)
    elif df.iloc[i,8] == 'million tonnes':
        quantity.append(df.iloc[i,7]*1000000)
    elif df.iloc[i,8] == 'million carats':
        quantity.append(df.iloc[i,7]*0.2)
    elif df.iloc[i,8] == 'tonnes':
        quantity.append(df.iloc[i,7])
    elif df.iloc[i,8] == '000 tonnes':
        quantity.append(df.iloc[i,7]*1.016)
    else:
        quantity.append('NaN')
        
        

In [18]:
dictionary =pd.DataFrame({'quantity(tonne)': quantity})
df = pd.concat([df, dictionary], axis=1)

In [19]:
df.head(5)

,SiteCode,StartDate,EndDate,Tonnage (Mt),Grade,GradeUnit,CommodityName,ContainedMetal,ContainedMetalUnit,CutOffGrade,TargetCommodityGroup,quantity(tonne)
0,S0000001,13/07/2005,14/09/2006,0.695,1.30,gram/tonne,Gold,903.50,kilogram,NaN,GOLD,0.9035
1,S0000002,30/06/1999,01/10/2003,0.179,2.12,gram/tonne,Gold,379.48,kilogram,NaN,GOLD,0.37948
2,S0000002,04/06/2003,06/02/2017,0.026,4.16,gram/tonne,Gold,108.16,kilogram,NaN,GOLD,0.10816
3,S0000002,04/06/2003,06/02/2017,0.137,2.64,gram/tonne,Gold,361.68,kilogram,NaN,GOLD,0.36168
4,S0000002,07/02/2017,11/03/2019,0.300,2.25,gram/tonne,Gold,675.00,kilogram,0.5,GOLD,0.675


In [20]:
df.GradeUnit.value_counts()

percent           36869
gram/tonne        20171
kilogram/tonne      278
carat/tonne         215
Name: GradeUnit, dtype: int64

### Filtering site code by date (most recent start date)

In [21]:
df.groupby('SiteCode')['StartDate'].unique()

SiteCode
S0000001                                        [13/07/2005]
S0000002    [30/06/1999, 04/06/2003, 07/02/2017, 12/03/2019]
S0000005                                        [30/09/2006]
S0000006                                        [08/10/1993]
S0000007    [26/09/1996, 21/10/2015, 26/11/2018, 01/11/2019]
                                  ...                       
S0239309                                        [07/09/2020]
S0239360                [16/03/2020, 08/03/2016, 08/03/2021]
S0239370                                        [09/03/2021]
S0239458                                        [31/12/2020]
S0239459                                        [31/12/2020]
Name: StartDate, Length: 4475, dtype: object

In [22]:
df.groupby('SiteCode')['EndDate'].unique()

SiteCode
S0000001                                 [14/09/2006]
S0000002    [01/10/2003, 06/02/2017, 11/03/2019, nan]
S0000005                                 [07/03/2016]
S0000006                                 [30/12/2014]
S0000007    [26/09/1997, 25/11/2018, 31/10/2019, nan]
                              ...                    
S0239309                                 [25/02/2021]
S0239360                [07/03/2021, 16/03/2020, nan]
S0239370                                        [nan]
S0239458                                        [nan]
S0239459                                        [nan]
Name: EndDate, Length: 4475, dtype: object

In [23]:
dict1 = dict(df.groupby('SiteCode')['StartDate'].unique())

In [24]:
# Create dictionary includes site code with the most recent start date
dict2 = {}
for i in dict1:
    dict2[i]=dict1[i][-1]

In [25]:
# Create Dataframe with site code and the most recent start date
data =pd.DataFrame()
for i in dict2:
    df1=df[df['SiteCode'].str.contains(i)]
    df2=df1[df1['StartDate'].str.contains(dict2[i])]
    data = data.append(df2)

In [26]:
data

,SiteCode,StartDate,EndDate,Tonnage (Mt),Grade,GradeUnit,CommodityName,ContainedMetal,ContainedMetalUnit,CutOffGrade,TargetCommodityGroup,quantity(tonne)
0,S0000001,13/07/2005,14/09/2006,0.695,1.30,gram/tonne,Gold,903.50,kilogram,NaN,GOLD,0.9035
6,S0000002,12/03/2019,NaN,0.528,2.09,gram/tonne,Gold,1103.52,kilogram,0.3,GOLD,1.10352
7,S0000005,30/09/2006,07/03/2016,0.011,7.33,gram/tonne,Gold,80.63,kilogram,NaN,GOLD,0.08063
8,S0000005,30/09/2006,07/03/2016,0.018,4.94,gram/tonne,Gold,88.92,kilogram,NaN,GOLD,0.08892
9,S0000006,08/10/1993,30/12/2014,0.057,2.36,gram/tonne,Gold,134.52,kilogram,NaN,GOLD,0.13452
...,...,...,...,...,...,...,...,...,...,...,...,...
57530,S0239459,31/12/2020,NaN,0.000,2.20,gram/tonne,Gold,0.00,kilogram,2.0,GOLD,0.0
57531,S0239459,31/12/2020,NaN,4.000,0.30,gram/tonne,Gold,1200.00,kilogram,0.4,GOLD,1.2
57532,S0239459,31/12/2020,NaN,25.000,1.20,gram/tonne,Gold,30000.00,kilogram,0.6,GOLD,30.0
57533,S0239459,31/12/2020,NaN,2.000,3.30,gram/tonne,Gold,6600.00,kilogram,2.0,GOLD,6.6


In [32]:
# There are 4 unit measures in the GradeUnit column
# 'gram/tonne' is equal to ppm
data.GradeUnit.unique()

array(['gram/tonne', 'percent', 'kilogram/tonne', 'carat/tonne'],
      dtype=object)

In [33]:
data1 = data[data['GradeUnit']=='gram/tonne']

In [34]:
data2 = data[data['GradeUnit']=='percent']

In [35]:
data3 = data[data['GradeUnit']=='kilogram/tonne']

In [36]:
data4 = data[data['GradeUnit']=='carat/tonne']

Create lists of ppm for each subseted data

In [ ]:
list_of_ppm_data2=[]
for i in range(len(data2)):
    list_of_ppm_data2.append((data2.iloc[i,11]*1000000)/(data2.iloc[i,3]*1000000))

In [ ]:
list_of_ppm_data3=[]
for i in range(len(data3)):
    list_of_ppm_data3.append((data3.iloc[i,11]*1000000)/(data3.iloc[i,3]*1000000))

In [ ]:
list_of_ppm_data4=[]
for i in range(len(data4)):
    list_of_ppm_data4.append((data4.iloc[i,11]*1000000)/(data4.iloc[i,3]*1000000))

When transform from percent to ppm, the ppm values increase dramatically

In [ ]:
# Append column ppm to the dataframe
data1['ppm'] = list(data1['Grade'])
data2['ppm'] = list_of_ppm_data2
data3['ppm'] = list_of_ppm_data3
data4['ppm'] = list_of_ppm_data4

In [30]:
filtered_data = pd.DataFrame()
filtered_data = filtered_data.append(data1)
filtered_data = filtered_data.append(data2)
filtered_data = filtered_data.append(data3)
filtered_data = filtered_data.append(data4)

In [35]:
filtered_data.head(10)

,SiteCode,StartDate,EndDate,Tonnage (Mt),Grade,GradeUnit,CommodityName,ContainedMetal,ContainedMetalUnit,CutOffGrade,TargetCommodityGroup,quantity(tonne),ppm
0,S0000001,13/07/2005,14/09/2006,0.6950,1.30,gram/tonne,Gold,903.500,kilogram,NaN,GOLD,0.9035,1.30
6,S0000002,12/03/2019,NaN,0.5280,2.09,gram/tonne,Gold,1103.520,kilogram,0.3,GOLD,1.10352,2.09
7,S0000005,30/09/2006,07/03/2016,0.0110,7.33,gram/tonne,Gold,80.630,kilogram,NaN,GOLD,0.08063,7.33
8,S0000005,30/09/2006,07/03/2016,0.0180,4.94,gram/tonne,Gold,88.920,kilogram,NaN,GOLD,0.08892,4.94
9,S0000006,08/10/1993,30/12/2014,0.0570,2.36,gram/tonne,Gold,134.520,kilogram,NaN,GOLD,0.13452,2.36
10,S0000006,08/10/1993,30/12/2014,0.0340,2.16,gram/tonne,Gold,73.440,kilogram,NaN,GOLD,0.07344,2.16
19,S0000007,01/11/2019,NaN,0.4039,1.27,gram/tonne,Gold,512.953,kilogram,0.6,GOLD,0.512953,1.27
20,S0000007,01/11/2019,NaN,3.3684,1.10,gram/tonne,Gold,3705.240,kilogram,0.6,GOLD,3.70524,1.10
21,S0000007,01/11/2019,NaN,0.4618,0.79,gram/tonne,Gold,364.822,kilogram,0.6,GOLD,0.364822,0.79
36,S0000008,15/04/2021,NaN,4.7872,1.57,gram/tonne,Gold,7515.904,kilogram,0.6,GOLD,7.515904,1.57


In [34]:
filtered_data.to_csv('Transformed_Minedex_data.csv',index=False)

In [9]:
data = pd.read_csv('Transformed_Minedex_data.csv')